In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("JupyterTest") \
    .master("spark://spark-master:7077") \
    .getOrCreate()
spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/15 03:22:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark = SparkSession.builder \
    .appName("JupyterTest") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

try:
    print("Trying to read from Kafka topic 'farm_raw_telemetry'...")
    
    df = (
        spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "broker1:29092")  # match your Docker hostname & port
        .option("subscribe", "farm_raw_telemetry")
        .option("startingOffsets", "latest")  # only read new messages
        .load()
    )

    # --- 3. Show the schema ---
    df.printSchema()

    # --- 4. Display messages in console ---
    query = (
        df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
        .writeStream
        .outputMode("append")
        .format("console")
        .option("truncate", "false")
        .start()
    )

    query.awaitTermination(30)  # stream for 30 seconds then stop
    query.stop()

except Exception as e:
    print("❌ Kafka connection failed:", e)
finally:
    spark.stop()
    print("Done")

Trying to read from Kafka topic 'farm_raw_telemetry'...
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



25/11/15 03:30:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1f3395ed-643b-4b22-bbf4-4c6c801ebf64. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/11/15 03:30:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
+---+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                         |
+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NULL|{"alert_info": {"farm_id": "farm0", "gateway_id": "gw0-1", "device_id": "farm0-gw1-dev1", "ts": 1763177411253, "ts_iso": "2025-11-15T03:30:20.407948+00:00", "new_state": "OK", "details": []}}                

-------------------------------------------
Batch: 2
-------------------------------------------
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                                                                |
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NULL|{"farm_id": "farm0", "gateway_id": "gw0-0"

In [13]:
spark.stop()

In [ ]:
try:
    # small test dataframe
    df = spark.createDataFrame([{"id": 1, "message": "Hello Elasticsearch"}])

    df.write \
      .format("org.elasticsearch.spark.sql") \
      .option("es.nodes", "elasticsearch") \
      .option("es.port", "9200") \
      .option("es.resource", "farm_raw_telemetry") \
      .mode("overwrite") \
      .save()

    print("Elasticsearch write successful.")

except Exception as e:
    print("Elasticsearch connection failed:", e)

[Stage 0:=============================>                             (1 + 1) / 2]

Elasticsearch write successful.


In [3]:
import json
import argparse
import sqlite3
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, udf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType

In [4]:
conn = sqlite3.connect("/opt/locations.db")
cur = conn.cursor()
cur.execute("SELECT device_id, latitude, longitude, gateway_id, farm_id FROM devices")
rows = cur.fetchall()
conn.close()
loc_map = {r[0]: {"latitude": r[1], "longitude": r[2], "gateway_id": r[3], "farm_id": r[4]} for r in rows}

In [6]:
loc_map

{'farm0-gw0-dev0': {'latitude': 13.57306092631992,
  'longitude': 105.84890992866161,
  'gateway_id': 'gw0-0',
  'farm_id': 'farm0'},
 'farm0-gw0-dev1': {'latitude': 13.573239385900424,
  'longitude': 105.84969920486161,
  'gateway_id': 'gw0-0',
  'farm_id': 'farm0'},
 'farm0-gw0-dev2': {'latitude': 13.573446544291693,
  'longitude': 105.84934440703746,
  'gateway_id': 'gw0-0',
  'farm_id': 'farm0'},
 'farm0-gw0-dev3': {'latitude': 13.572307556647448,
  'longitude': 105.8482885011738,
  'gateway_id': 'gw0-0',
  'farm_id': 'farm0'},
 'farm0-gw0-dev4': {'latitude': 13.57066390588736,
  'longitude': 105.85091955310786,
  'gateway_id': 'gw0-0',
  'farm_id': 'farm0'},
 'farm0-gw0-dev5': {'latitude': 13.571951260005234,
  'longitude': 105.85007356004881,
  'gateway_id': 'gw0-0',
  'farm_id': 'farm0'},
 'farm0-gw0-dev6': {'latitude': 13.572881610626947,
  'longitude': 105.84922914642104,
  'gateway_id': 'gw0-0',
  'farm_id': 'farm0'},
 'farm0-gw0-dev7': {'latitude': 13.573973942307902,
  'lon